In [41]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import json


C:\Users\tashi\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [23]:
def scrape_vot_article(url):
    """
    Scrapes an article from the VOT (Voice of Tibet) website.

    Args:
    url (str): The URL of the VOT article to scrape.

    Returns:
    dict: A dictionary containing the scraped information with the following structure:
        {
            'title': str,
            'body': {
                'Audio': str,
                'Text': List[str]
            },
            'meta_data': {
                'Author': str,
                'Date': str,
                'Tags': List[str],
                'URL': str
            }
        }
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract title
        title = soup.find('h1', class_='tdb-title-text')
        title_text = title.text.strip() if title else "Title not found"

        # Extracting Meta Data
        meta_data = {"URL": url}
        try:
            meta_data_body = soup.find('div', class_="vc_column_inner tdi_85 wpb_column vc_column_container tdc-inner-column td-pb-span6")
            if meta_data_body:
                author_name = meta_data_body.find('a', class_="tdb-author-name")
                meta_data["Author"] = author_name.get_text() if author_name else "Author not found"
                
                date_time = meta_data_body.find('time', class_="entry-date updated td-module-date")
                meta_data["Date"] = date_time.get_text() if date_time else "Date not found"
        except AttributeError:
            meta_data["Author"] = "Error fetching author"
            meta_data["Date"] = "Error fetching date"

        # Getting tag meta data 
        tag_list_meta = []
        try:
            tag_meta = soup.find('ul', class_='tdb-tags')
            if tag_meta:
                tag_meta = tag_meta.select('li a')
                tag_list_meta = [tag.text for tag in tag_meta]
        except AttributeError:
            pass
        meta_data["Tags"] = tag_list_meta

        # Extract body content
        body_text = {"Audio": "No audio found", "Text": []}
        try:
            body = soup.find('div', class_='td_block_wrap tdb_single_content tdi_100 td-pb-border-top td_block_template_1 td-post-content tagdiv-type')
            if body:
                # Extracting all <p> tags for text content
                paragraphs = body.find_all('p')
                body_text["Text"] = [para.get_text(strip=True) for para in paragraphs]

                # Find the audio tag and get its src attribute
                audio = body.find('figure', class_='wp-block-audio')
                if audio:
                    audio_tag = audio.find('audio')
                    if audio_tag:
                        body_text["Audio"] = audio_tag.get('src', "No audio source found")
        except AttributeError:
            body_text["Text"] = ["Error fetching body content"]
        
        return {
            'title': title_text,
            'body': body_text,
            'meta_data': meta_data
        }
    except requests.RequestException as e:
        return {
            'title': "Error",
            'body': {"Audio": "Error", "Text": [f"An error occurred while fetching the article: {str(e)}"]},
            'meta_data': {'URL': url}
        }


In [29]:
# Usage
url = "https://vot.org/%e0%bd%96%e0%bd%bc%e0%bd%91%e0%bc%8b%e0%bd%80%e0%be%b1%e0%bd%b2%e0%bc%8b%e0%bd%82%e0%bd%9e%e0%bd%bc%e0%bd%93%e0%bc%8b%e0%bd%93%e0%bd%b4%e0%bc%8b%e0%bd%a3%e0%be%b7%e0%bd%93%e0%bc%8b%e0%bd%9a-20/"
article = scrape_vot_article(url)

article

{'title': 'བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་མཇུག་བསྒྲིལ་འདུག',
 'body': {'Audio': 'https://vot.org/wp-content/uploads/2024/08/Gangtok-TYC-54-Working-Committee-Meet-Concludes.mp3',
  'Text': ['ཁ་སྔོན་ཕྱི་ཚེས་ ༥ ནས་འགོ་འཛུགས་བྱུང་བའི་བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ དེ་ཕྱི་ཚེས་ ༡༠ ཉིན་མཇུག་བསྒྲིལ་ཡོད་པ་དང་།\xa0\xa0 བོད་ཀྱི་གཞོན་ནུ་ལྷན་ཚོགས་ཀྱི་ངོས་ནས་ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་འདིའི་བརྒྱུད།\xa0 རྒྱ་ནག་གཞུང་གིས་བོད་མིའི་ངོ་བོ་རྩ་མེད་བཟོ་བའི་སྲིད་བྱུས་འོག\xa0\xa0 བོད་ཀྱི་ཆོས་དང་རིག་གཞུང་སྐད་ཡིག་བཅས་པར་ཕྱོགས་གང་ས་ནས་དམ་དྲག་ཇེ་ཆེར་བྱེད་བཞིན་པ་དེར་ངོ་རྒོལ་གྱིས་བསྒྲགས་གཏམ་ཞིག་སྤེལ་ཡོད་པ་རེད།',
   'ད་རེས་ཀྱི་འགན་འཛིན་ལྷན་ཚོགས་ཐེངས་ ༥༤ པ་དེའི་ཐོག\xa0\xa0 རྒྱ་གར་དང་བལ་ཡུལ་བཅས་ས་གནས་ཁག་ ༣༨ ནས་ཚོགས་བཅར་བ་མི་གྲངས་\xa0 ༢༠༠ ཙམ་ཞིག་འདུ་འཛོམས་བྱུང་ཡོད་པ་དང་།\xa0 ཉིན་གྲངས་ལྔའི་རིང་ཚོགས་བཅར་བ་ཕན་ཚུན་བཀའ་བསྡུར་དང་འབྲེལ་གྲོས་ཆོད་དོན་ཚན་བརྒྱད་གཏན་འབེབས་གནང་ཐུབ་པ་མ་ཟད།\xa0 སྤྱི་མོས་གྲོས་ཆོད་ཞིག་གཏན་འབེབས་བྱུང་ཡོད་པ་དེའི་ནང་བོད་ཀྱི་སྐད་ཡིག་མི་ཉམས་རྒྱུན་འཛིན་སླད།

In [27]:
url = "https://vot.org/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%a8%e0%bc%8b%e0%bd%a2%e0%bd%b2%e0%bd%a0%e0%bd%b2%e0%bc%8b%e0%bd%93%e0%bd%ba%e0%bd%a0/"
article = scrape_vot_article(url)


article

{'title': '༸གོང་ས་མཆོག་ཨ་རིའི་ནེའུ་ཡོག་གྲོང་ཁྱེར་ནང་༸ཞབས་སོར་བདེ་བར་འཁོད་པ།',
 'body': {'Audio': 'No audio found',
  'Text': ['བོད་ཀྱི་བླ་ན་མེད་པའི་དབུ་ཁྲིད་སྤྱི་ནོར་༸གོང་ས་༸སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ལྷོ་ California མངའ་སྡེ་ Anaheim གྲོང་ཁྱེར་དུ་ཉིན་གསུམ་རིང་གོ་ལའི་བྱམས་བརྩེའི་ལྷན་ཚོགས་ཀྱི་མཛད་འཕྲིན་ཁག་ལེགས་པར་གྲུབ་པ་དང། ཟླ་འདིའི་ཕྱི་ཚེས་ ༨ ཀྱི་ཕྱི་དྲོ་ཚོུད་ ༧།༣༠ ཡོལ་ཙམ་ལ་ཨ་རིའི་གྲོང་ཁྱེར་ནེའུ་ཡོག་ནང་༸ཞབས་སོར་བདེ་བར་འཁོད་གནང་མཛད་འདུགམགོན་པོ་༸གང་ཉིད་མཆོག་གི་བཞུགས་གནས་ Park hyde hotel གསོལ་མགྲོན་ཁང་དུ་བོད་མིའི་སྒྲིག་འཛུགས་ཀྱི་སྲིད་སྐྱོང་བློ་བཟང་སེང་གེ་མཆོག་དང། བོད་མི་མང་སྤྱི་འཐུས་ལྷན་ཚོགས་ཀྱི་ཚོགས་གཙོ་སྤེན་པ་ཚེ་རིང་མཆོག ཆོས་རིག་བཀའ་བློན་པདྨ་ཆོས་འབྱོར་མཆོག་དབུས་པས། བྱང་ཨ་རིའི་སྤྱི་འཐུས། བོད་རིགས་སྤྱི་མཐུན་ཚོགས་པའི་ཚོགས་གཙོ་དང། གཞན་ཡང་བོད་མི་སེར་སྐྱ་མང་ཚོགས་ཚོས་དགའ་དད་སྤྲོ་གསུམ་གྱི་ངང་ནས་བོད་ལུགས་ལྟར་གྲོ་སོ་ཕྱེ་མར་འདེགས་འབུལ་ཞུས་ནས་ཕེབས་བསུ་ཞུས་འདུགཟླ་འདིའི་ཕྱི་ཚེས་ ༩ ཉིན་༸གོང་ས་སྐྱབས་མགོན་ཆེན་པོ་མཆོག་ནས་ནེའུ་ཡོག་གྲོང་ཁྱེར་ནང་རྟེན་གཞི་བྱས་པའི་ Javits Center ནང་དད་ལྡན་སེར་སྐྱ་མང་ཚོགས་ ༡༤༠༠༠ ལྷ

## Extract All Artical Links from all page 

In [31]:
def extract_all_article(url: str):
    """
    Extracts all article links from a given VOT (Voice of Tibet) webpage.

    This function scrapes the provided URL and extracts links to individual articles
    found on the page.

    Args:
    url (str): The URL of the VOT webpage containing article links.

    Returns:
    List[str]: A list of URLs to individual articles.

    Raises:
    requests.RequestException: If there's an error fetching the webpage.
    ValueError: If the expected HTML structure is not found on the page.
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        all_article = soup.find("div", class_="td_block_inner tdb-block-inner td-fix-index")
        if not all_article:
            raise ValueError("Could not find the main article container on the page.")
        
        article_links = all_article.find_all("a", class_="td-image-wrap")
        all_links = [link.get("href") for link in article_links if link.get("href")]
        
        return all_links

    except requests.RequestException as e:
        print(f"An error occurred while fetching the webpage: {e}")
        return []
    except ValueError as e:
        print(f"An error occurred while parsing the webpage: {e}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []


## Getting all links of གོང་ས་མཆོག from VOT
Base link : "https://vot.org/category/%e0%bd%96%e0%bd%bc%e0%bd%91%e0%bc%8d/page/" + i + "/"

In [49]:

initial_url = "https://vot.org/category/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82/"
# All_url_link = []
found_url_link = extract_all_article(initial_url)
All_url_link = {}
key = "Page གོང་ས་མཆོག " + str(1)
All_url_link[key] = found_url_link



In [50]:

custom_url = "https://vot.org/category/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82/page/"
for i in tqdm(range(2,214)):
    final_url = custom_url + str(i) +"/"
    found_url_link = extract_all_article(final_url)
    key = "Page གོང་ས་མཆོག " + str(i)
    All_url_link[key] = found_url_link
    # All_url_link.extend(found_url_link)

100%|██████████| 212/212 [08:08<00:00,  2.30s/it]


In [51]:
All_url_link

{'Page གོང་ས་མཆོག 1': ['https://vot.org/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%91%e0%bd%ba%e0%bc%8b%e0%bd%a2%e0%bd%b2%e0%bd%84%e0%bc%8b%e0%bd%95%e0%be%b1%e0%bd%b2/',
  'https://vot.org/%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%82%e0%bd%b2%e0%bd%a6%e0%bc%8b-swiss-%e0%bd%91%e0%bd%84%e0%bc%8b-liechtenstein/',
  'https://vot.org/%e0%bd%a8%e0%bc%8b%e0%bd%a2%e0%bd%b2%e0%bd%a0%e0%bd%b2%e0%bc%8b%e0%bd%93%e0%bd%b2%e0%bd%a0%e0%bd%b4%e0%bc%8b%e0%bd%a1%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%82%e0%bd%b2%e0%bc%8b-ubs-arena-%e0%bd%a2/',
  'https://vot.org/vot%e0%bd%82%e0%bd%84%e0%bd%a6%e0%bd%98%e0%bd%86%e0%bd%82%e0%bd%a3%e0%bd%96%e0%bd%a2%e0%bd%93%e0%bd%96%e0%bd%9e-2/',
  'https://vot.org/wisdom-of-happiness-%e0%bd%9e%e0%bd%ba%e0%bd%a6%e0%bc%8b%e0%bc%b8%e0%bd%82%e0%bd%bc%e0%bd%84%e0%bc%8b%e0%bd%a6%e0%bc%8b%e0%bd%98%e0%bd%86%e0%bd%bc%e0%bd%82%e0%bc%8b%e0%bd%91%

In [62]:
print(f"Total page processed: {len(All_url_link)}")

Total page processed: 213


## Saving it as Json File

In [59]:
with open("ALL_link_གོང་ས་མཆོག.json", "w") as outfile:
    json.dump(All_url_link , outfile, indent=4)

བོད།

In [60]:
with open('ALL_link_གོང་ས་མཆོག.json', 'r') as openfile:
 
    # Reading from json file
    Loaded_All_url_link = json.load(openfile)